In [1]:
import os
import scipy.misc
import numpy as np
import json

from model_steer import DCGAN
from utils import pp, visualize, to_json, show_all_variables, expand_path, timestamp

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 25, "Epoch to train [25]")
flags.DEFINE_float("learning_rate", 0.0002, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_float("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_integer("input_height", 108, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", None, "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", None, "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "mnist", "The name of dataset [celebA, mnist, lsun]")
flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("data_dir", "./data", "path to datasets [e.g. $HOME/data]")
flags.DEFINE_string("out_dir", "./out", "Root directory for outputs [e.g. $HOME/out]")
flags.DEFINE_string("out_name", "", "Folder (under out_root_dir) for all outputs. Generated automatically if left blank []")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Folder (under out_root_dir/out_name) to save checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Folder (under out_root_dir/out_name) to save samples [samples]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("visualize", False, "True for visualizing, False for nothing [False]")
flags.DEFINE_boolean("export", False, "True for exporting with new batch size")
flags.DEFINE_boolean("freeze", False, "True for exporting with new batch size")
flags.DEFINE_integer("max_to_keep", 1, "maximum number of checkpoints to keep")
flags.DEFINE_integer("sample_freq", 200, "sample every this many iterations")
flags.DEFINE_integer("ckpt_freq", 200, "save checkpoint every this many iterations")
flags.DEFINE_integer("z_dim", 100, "dimensions of z")
flags.DEFINE_string("z_dist", "uniform_signed", "'normal01' or 'uniform_unsigned' or uniform_signed")
flags.DEFINE_boolean("G_img_sum", False, "Save generator image summaries in log")
#flags.DEFINE_integer("generate_test_images", 100, "Number of images to generate during test. [100]")
FLAGS = flags.FLAGS

In [2]:
def main(_):
  pp.pprint(flags.FLAGS.__flags)
  
  # expand user name and environment variables
  FLAGS.data_dir = expand_path(FLAGS.data_dir)
  FLAGS.out_dir = expand_path(FLAGS.out_dir)
  FLAGS.out_name = expand_path(FLAGS.out_name)
  FLAGS.checkpoint_dir = expand_path(FLAGS.checkpoint_dir)
  FLAGS.sample_dir = expand_path(FLAGS.sample_dir)

  print('FLAGS.dataset:', FLAGS.dataset)  
  print('FLAGS.output_height', FLAGS.output_height)  
  print('flags.train', FLAGS.train)

  if FLAGS.output_height is None: FLAGS.output_height = FLAGS.input_height
  if FLAGS.input_width is None: FLAGS.input_width = FLAGS.input_height
  if FLAGS.output_width is None: FLAGS.output_width = FLAGS.output_height

  # output folders
  if FLAGS.out_name == "":
      FLAGS.out_name = '{} - {} - {}'.format(timestamp(), FLAGS.data_dir.split('/')[-1], FLAGS.dataset) # penultimate folder of path
      if FLAGS.train:
        FLAGS.out_name += ' - x{}.z{}.{}.y{}.b{}'.format(FLAGS.input_width, FLAGS.z_dim, FLAGS.z_dist, FLAGS.output_width, FLAGS.batch_size)

  FLAGS.out_dir = os.path.join(FLAGS.out_dir, FLAGS.out_name)
  FLAGS.checkpoint_dir = os.path.join(FLAGS.out_dir, FLAGS.checkpoint_dir)
  FLAGS.sample_dir = os.path.join(FLAGS.out_dir, FLAGS.sample_dir)

  if not os.path.exists(FLAGS.checkpoint_dir): os.makedirs(FLAGS.checkpoint_dir)
  if not os.path.exists(FLAGS.sample_dir): os.makedirs(FLAGS.sample_dir)

  with open(os.path.join(FLAGS.out_dir, 'FLAGS.json'), 'w') as f:
    flags_dict = {k:FLAGS[k].value for k in FLAGS}
    json.dump(flags_dict, f, indent=4, sort_keys=True, ensure_ascii=False)
  

  #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
  run_config = tf.ConfigProto()
  run_config.gpu_options.allow_growth=True

  with tf.Session(config=run_config) as sess:
    if FLAGS.dataset == 'mnist':
      dcgan = DCGAN(
          sess,
          input_width=FLAGS.input_width,
          input_height=FLAGS.input_height,
          output_width=FLAGS.output_width,
          output_height=FLAGS.output_height,
          batch_size=FLAGS.batch_size,
          sample_num=FLAGS.batch_size,
          y_dim=10,
          z_dim=FLAGS.z_dim,
          dataset_name=FLAGS.dataset,
          input_fname_pattern=FLAGS.input_fname_pattern,
          crop=FLAGS.crop,
          checkpoint_dir=FLAGS.checkpoint_dir,
          sample_dir=FLAGS.sample_dir,
          data_dir=FLAGS.data_dir,
          out_dir=FLAGS.out_dir,
          max_to_keep=FLAGS.max_to_keep)
    else:
      dcgan = DCGAN(
          sess,
          input_width=FLAGS.input_width,
          input_height=FLAGS.input_height,
          output_width=FLAGS.output_width,
          output_height=FLAGS.output_height,
          batch_size=FLAGS.batch_size,
          sample_num=FLAGS.batch_size,
          z_dim=FLAGS.z_dim,
          dataset_name=FLAGS.dataset,
          input_fname_pattern=FLAGS.input_fname_pattern,
          crop=FLAGS.crop,
          checkpoint_dir=FLAGS.checkpoint_dir,
          sample_dir=FLAGS.sample_dir,
          data_dir=FLAGS.data_dir,
          out_dir=FLAGS.out_dir,
          max_to_keep=FLAGS.max_to_keep)

    show_all_variables()

    if FLAGS.train:
      dcgan.train(FLAGS)
    else:
      load_success, load_counter = dcgan.load(FLAGS.checkpoint_dir)
      if not load_success:
        raise Exception("Checkpoint not found in " + FLAGS.checkpoint_dir)


    # to_json("./web/js/layers.js", [dcgan.h0_w, dcgan.h0_b, dcgan.g_bn0],
    #                 [dcgan.h1_w, dcgan.h1_b, dcgan.g_bn1],
    #                 [dcgan.h2_w, dcgan.h2_b, dcgan.g_bn2],
    #                 [dcgan.h3_w, dcgan.h3_b, dcgan.g_bn3],
    #                 [dcgan.h4_w, dcgan.h4_b, None])

    # Below is codes for visualization
      if FLAGS.export:
        export_dir = os.path.join(FLAGS.checkpoint_dir, 'export_b'+str(FLAGS.batch_size))
        dcgan.save(export_dir, load_counter, ckpt=True, frozen=False)

      if FLAGS.freeze:
        export_dir = os.path.join(FLAGS.checkpoint_dir, 'frozen_b'+str(FLAGS.batch_size))
        dcgan.save(export_dir, load_counter, ckpt=False, frozen=True)

      if FLAGS.visualize:
        OPTION = 1
        visualize(sess, dcgan, FLAGS, OPTION, FLAGS.sample_dir)

# if __name__ == '__main__':
#   tf.app.run()

In [3]:
tf.app.run(argv=['--dataset', 'mnist', '--input_height=28', '--output_height=28', '--train'])

{'G_img_sum': <absl.flags._flag.BooleanFlag object at 0x7f5cf10d2668>,
 'batch_size': <absl.flags._flag.Flag object at 0x7f5dc9afc160>,
 'beta1': <absl.flags._flag.Flag object at 0x7f5dc9afc080>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x7f5cf10caf28>,
 'ckpt_freq': <absl.flags._flag.Flag object at 0x7f5cf10d2390>,
 'crop': <absl.flags._flag.BooleanFlag object at 0x7f5cf10d2160>,
 'data_dir': <absl.flags._flag.Flag object at 0x7f5cf10cad68>,
 'dataset': <absl.flags._flag.Flag object at 0x7f5cf10cabe0>,
 'epoch': <absl.flags._flag.Flag object at 0x7f5dc82c59e8>,
 'export': <absl.flags._flag.BooleanFlag object at 0x7f5cf10d2278>,
 'freeze': <absl.flags._flag.BooleanFlag object at 0x7f5cf10d22e8>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x7f5de42a04a8>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x7f5de42a04a8>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x7f5de42a04e0>,
 'helpshort': <tensorflow.python.platform.app

[      47 Epoch:[ 0/25] [  46/1093] time: 6.9722, d_loss: 1.50945091, g_loss: 2.23329639
[      48 Epoch:[ 0/25] [  47/1093] time: 7.0401, d_loss: 1.53168607, g_loss: 2.26761818
[      49 Epoch:[ 0/25] [  48/1093] time: 7.1105, d_loss: 1.52177358, g_loss: 2.23154593
[      50 Epoch:[ 0/25] [  49/1093] time: 7.1829, d_loss: 1.53026986, g_loss: 2.23109818
[      51 Epoch:[ 0/25] [  50/1093] time: 7.2507, d_loss: 1.52481437, g_loss: 2.26755476
[      52 Epoch:[ 0/25] [  51/1093] time: 7.3194, d_loss: 1.51674986, g_loss: 2.25353789
[      53 Epoch:[ 0/25] [  52/1093] time: 7.3830, d_loss: 1.52121615, g_loss: 2.26480913
[      54 Epoch:[ 0/25] [  53/1093] time: 7.4477, d_loss: 1.50476086, g_loss: 2.24162245
[      55 Epoch:[ 0/25] [  54/1093] time: 7.5190, d_loss: 1.50062895, g_loss: 2.24279881
[      56 Epoch:[ 0/25] [  55/1093] time: 7.5887, d_loss: 1.49120426, g_loss: 2.20899057
[      57 Epoch:[ 0/25] [  56/1093] time: 7.6557, d_loss: 1.48598289, g_loss: 2.21863437
[      58 Epoch:[ 0/2

[     140 Epoch:[ 0/25] [ 139/1093] time: 13.1163, d_loss: 1.48595107, g_loss: 2.14583874
[     141 Epoch:[ 0/25] [ 140/1093] time: 13.1857, d_loss: 1.47467875, g_loss: 2.15953708
[     142 Epoch:[ 0/25] [ 141/1093] time: 13.2547, d_loss: 1.47947562, g_loss: 2.22697735
[     143 Epoch:[ 0/25] [ 142/1093] time: 13.3241, d_loss: 1.49157810, g_loss: 2.17252517
[     144 Epoch:[ 0/25] [ 143/1093] time: 13.3954, d_loss: 1.48845780, g_loss: 2.19224453
[     145 Epoch:[ 0/25] [ 144/1093] time: 13.4577, d_loss: 1.48901701, g_loss: 2.16716480
[     146 Epoch:[ 0/25] [ 145/1093] time: 13.5271, d_loss: 1.48767281, g_loss: 2.14112186
[     147 Epoch:[ 0/25] [ 146/1093] time: 13.5971, d_loss: 1.47786176, g_loss: 2.14472008
[     148 Epoch:[ 0/25] [ 147/1093] time: 13.6626, d_loss: 1.48895979, g_loss: 2.20650530
[     149 Epoch:[ 0/25] [ 148/1093] time: 13.7292, d_loss: 1.50009847, g_loss: 2.13538790
[     150 Epoch:[ 0/25] [ 149/1093] time: 13.7958, d_loss: 1.48866761, g_loss: 2.10091758
[     151 

[     234 Epoch:[ 0/25] [ 233/1093] time: 23.6909, d_loss: 1.47143042, g_loss: 2.15782881
[     235 Epoch:[ 0/25] [ 234/1093] time: 23.7552, d_loss: 1.48246002, g_loss: 2.14560008
[     236 Epoch:[ 0/25] [ 235/1093] time: 23.8221, d_loss: 1.48119664, g_loss: 2.17932081
[     237 Epoch:[ 0/25] [ 236/1093] time: 23.8842, d_loss: 1.47044444, g_loss: 2.20968771
[     238 Epoch:[ 0/25] [ 237/1093] time: 23.9486, d_loss: 1.48497963, g_loss: 2.14615154
[     239 Epoch:[ 0/25] [ 238/1093] time: 24.0233, d_loss: 1.48671985, g_loss: 2.15499878
[     240 Epoch:[ 0/25] [ 239/1093] time: 24.0904, d_loss: 1.47353125, g_loss: 2.19591689
[     241 Epoch:[ 0/25] [ 240/1093] time: 24.1586, d_loss: 1.49397767, g_loss: 2.21054435
[     242 Epoch:[ 0/25] [ 241/1093] time: 24.2275, d_loss: 1.48587835, g_loss: 2.16991997
[     243 Epoch:[ 0/25] [ 242/1093] time: 24.2915, d_loss: 1.49098647, g_loss: 2.21028948
[     244 Epoch:[ 0/25] [ 243/1093] time: 24.3554, d_loss: 1.48776436, g_loss: 2.17831016
[     245 

[     328 Epoch:[ 0/25] [ 327/1093] time: 29.7236, d_loss: 1.48099518, g_loss: 2.16741371
[     329 Epoch:[ 0/25] [ 328/1093] time: 29.7918, d_loss: 1.47629356, g_loss: 2.23502398
[     330 Epoch:[ 0/25] [ 329/1093] time: 29.8567, d_loss: 1.49311304, g_loss: 2.18148327
[     331 Epoch:[ 0/25] [ 330/1093] time: 29.9266, d_loss: 1.48821795, g_loss: 2.21859932
[     332 Epoch:[ 0/25] [ 331/1093] time: 29.9968, d_loss: 1.48355865, g_loss: 2.19496393
[     333 Epoch:[ 0/25] [ 332/1093] time: 30.0627, d_loss: 1.47774386, g_loss: 2.26398706
[     334 Epoch:[ 0/25] [ 333/1093] time: 30.1322, d_loss: 1.47529387, g_loss: 2.20690823
[     335 Epoch:[ 0/25] [ 334/1093] time: 30.2021, d_loss: 1.49682426, g_loss: 2.21280289
[     336 Epoch:[ 0/25] [ 335/1093] time: 30.2635, d_loss: 1.49170804, g_loss: 2.15381765
[     337 Epoch:[ 0/25] [ 336/1093] time: 30.3247, d_loss: 1.46669078, g_loss: 2.15755796
[     338 Epoch:[ 0/25] [ 337/1093] time: 30.3876, d_loss: 1.46077061, g_loss: 2.13415933
[     339 

...

[   27087 Epoch:[24/25] [ 854/1093] time: 2279.6865, d_loss: 1.34638011, g_loss: 2.22079754
[   27088 Epoch:[24/25] [ 855/1093] time: 2279.7480, d_loss: 1.33885527, g_loss: 2.35050797
[   27089 Epoch:[24/25] [ 856/1093] time: 2279.8092, d_loss: 1.32236648, g_loss: 2.29707003
[   27090 Epoch:[24/25] [ 857/1093] time: 2279.8702, d_loss: 1.36972976, g_loss: 2.31316710
[   27091 Epoch:[24/25] [ 858/1093] time: 2279.9323, d_loss: 1.36914873, g_loss: 2.26281595
[   27092 Epoch:[24/25] [ 859/1093] time: 2279.9937, d_loss: 1.35373139, g_loss: 2.30835915
[   27093 Epoch:[24/25] [ 860/1093] time: 2280.0540, d_loss: 1.32001996, g_loss: 2.28933191
[   27094 Epoch:[24/25] [ 861/1093] time: 2280.1160, d_loss: 1.33164692, g_loss: 2.34652972
[   27095 Epoch:[24/25] [ 862/1093] time: 2280.1779, d_loss: 1.34336507, g_loss: 2.24130869
[   27096 Epoch:[24/25] [ 863/1093] time: 2280.2403, d_loss: 1.30759764, g_loss: 2.31381464
[   27097 Epoch:[24/25] [ 864/1093] time: 2280.3027, d_loss: 1.29001045, g_loss:

[   27179 Epoch:[24/25] [ 946/1093] time: 2285.6794, d_loss: 1.36804843, g_loss: 2.22542334
[   27180 Epoch:[24/25] [ 947/1093] time: 2285.7504, d_loss: 1.30976963, g_loss: 2.37324572
[   27181 Epoch:[24/25] [ 948/1093] time: 2285.8187, d_loss: 1.33646154, g_loss: 2.27410460
[   27182 Epoch:[24/25] [ 949/1093] time: 2285.8877, d_loss: 1.33100343, g_loss: 2.28569365
[   27183 Epoch:[24/25] [ 950/1093] time: 2285.9538, d_loss: 1.29134429, g_loss: 2.41118121
[   27184 Epoch:[24/25] [ 951/1093] time: 2286.0190, d_loss: 1.34880471, g_loss: 2.15952253
[   27185 Epoch:[24/25] [ 952/1093] time: 2286.0810, d_loss: 1.31784534, g_loss: 2.24755740
[   27186 Epoch:[24/25] [ 953/1093] time: 2286.1439, d_loss: 1.33903742, g_loss: 2.27095890
[   27187 Epoch:[24/25] [ 954/1093] time: 2286.2118, d_loss: 1.32557368, g_loss: 2.19335556
[   27188 Epoch:[24/25] [ 955/1093] time: 2286.2731, d_loss: 1.37979150, g_loss: 2.31093073
[   27189 Epoch:[24/25] [ 956/1093] time: 2286.3349, d_loss: 1.31865871, g_loss:

[   27270 Epoch:[24/25] [1037/1093] time: 2296.5982, d_loss: 1.34045434, g_loss: 2.21000767
[   27271 Epoch:[24/25] [1038/1093] time: 2296.6694, d_loss: 1.36349583, g_loss: 2.23749828
[   27272 Epoch:[24/25] [1039/1093] time: 2296.7413, d_loss: 1.36049008, g_loss: 2.39223504
[   27273 Epoch:[24/25] [1040/1093] time: 2296.8133, d_loss: 1.33471739, g_loss: 2.29640198
[   27274 Epoch:[24/25] [1041/1093] time: 2296.8828, d_loss: 1.35108936, g_loss: 2.31780076
[   27275 Epoch:[24/25] [1042/1093] time: 2296.9535, d_loss: 1.38350558, g_loss: 2.28977442
[   27276 Epoch:[24/25] [1043/1093] time: 2297.0218, d_loss: 1.30752027, g_loss: 2.33370113
[   27277 Epoch:[24/25] [1044/1093] time: 2297.0870, d_loss: 1.33654106, g_loss: 2.32675791
[   27278 Epoch:[24/25] [1045/1093] time: 2297.1520, d_loss: 1.29195082, g_loss: 2.19593215
[   27279 Epoch:[24/25] [1046/1093] time: 2297.2157, d_loss: 1.37284207, g_loss: 2.23652458
[   27280 Epoch:[24/25] [1047/1093] time: 2297.2794, d_loss: 1.31951308, g_loss:

SystemExit: 

/afs/csail.mit.edu/u/j/jahanian/.conda/envs/myenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print('loading mnist...')
data_dir = os.path.join('./data', 'mnist')

fd = open(os.path.join(data_dir,'train-images-idx3-ubyte'))
loaded = np.fromfile(file=fd,dtype=np.uint8)
trX = loaded[16:].reshape((60000,28,28,1)).astype(np.float)

idx = np.random.choice(60000, 5, replace=False)
for i in range(len(idx)):
    img_test_5px = np.zeros([28,28], dtype= 'float')
    img_test = trX[idx[i],:,:,0]
    offset_val = np.random.randint(1, 6)  
    coin = np.random.uniform(0, 1)
    if coin <= 0.5:
        offset_val = -offset_val            
    if(offset_val > 0):
        img_test_5px[:,offset_val:] = img_test[:,:-offset_val]
    else:
        img_test_5px[:,:28+offset_val] = img_test[:,-offset_val:]
    trX[idx[i],:,:,0] = img_test_5px
#     plt.figure()
#     plt.imshow(img_test_5px)

fd = open(os.path.join(data_dir,'train-labels-idx1-ubyte'))
loaded = np.fromfile(file=fd,dtype=np.uint8)
trY = loaded[8:].reshape((60000)).astype(np.float)

fd = open(os.path.join(data_dir,'t10k-images-idx3-ubyte'))
loaded = np.fromfile(file=fd,dtype=np.uint8)
teX = loaded[16:].reshape((10000,28,28,1)).astype(np.float)

idx = np.random.choice(10000, 9, replace=False)
for i in range(len(idx)):
    img_test_5px = np.zeros([28,28], dtype= 'float')
    img_test = teX[idx[i],:,:,0]
    offset_val = np.random.randint(0, 6)  
    coin = np.random.uniform(0, 1)
    if coin <= 0.5:
        offset_val = -offset_val            
    if(offset_val > 0):
        img_test_5px[:,offset_val:] = img_test[:,:-offset_val]
    else:
        img_test_5px[:,:28+offset_val] = img_test[:,-offset_val:]
    teX[idx[i],:,:,0] = img_test_5px
#     plt.figure()
#     plt.imshow(img_test_5px)


fd = open(os.path.join(data_dir,'t10k-labels-idx1-ubyte'))
loaded = np.fromfile(file=fd,dtype=np.uint8)
teY = loaded[8:].reshape((10000)).astype(np.float)